In [85]:
import pygame
import math
pygame.init()

screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

MVP
- FLOOR
- WALLS
- PLAYERS
- Collision mechanics
- RESOURCES
- RESOURCE SOURCE
- COUNTERTOPS:
    PLACING STUFF 
    GRAPHICS

FIX UP MANY THINGS IN ONE COUNTERTOP, KNIFE NOT DELETING ITSELF FOR SOME REASON

- CHOPPING BOARD
- FRYER
- Dishes
- CONVEYOR BELT

In [86]:
FLOOR_GRAPHIC = pygame.image.load('floor.png')
WALL_GRAPHIC = pygame.image.load('wall.png')
PLAYER1_GRAPHIC = pygame.image.load('player1.png')
PLAYER2_GRAPHIC = pygame.image.load('player2.png')
FISH_ICON = pygame.image.load('fish.png')
FISH_PLAIN = pygame.image.load('Plain_fish.png')
POTATO_ICON = pygame.image.load('potato.png')
POTATO_PLAIN = pygame.image.load('Plain_potato.png')
FISH_CRATE_ICON = pygame.image.load('FishCrate.png')
POTATO_CRATE_ICON = pygame.image.load('PotatoCrate.png')
COUNTERTOP_ICON = pygame.image.load('Countertop.png')
CB_ICON = pygame.image.load("CBoard.png")
CB_KNIFELESS_ICON = pygame.image.load("CB_knifeless.png")
CB_R_ICON = pygame.image.load("CBoard_rotated.png")
CB_R_KNIFELESS_ICON = pygame.image.load("CB_knifeless_rotated.png")



PLAYER_SPEED = 4

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)

In [87]:
SIZE = 32

class Object(pygame.sprite.Sprite):
    def __init__(self, position, graphic):
        super().__init__()
        self.image = graphic
        self.rect = self.image.get_rect(center=position)
        
    def check_distance(self,object2):
        x1, y1 = self.rect.centerx, self.rect.centery
        x2, y2 = object2.rect.centerx, object2.rect.centery
        return math.sqrt((x2-x1)**2 + (y2-y1)**2)
        
Floors = pygame.sprite.Group()
class Floor(Object):
    def __init__(self, position):
        super().__init__(position, FLOOR_GRAPHIC)
        Floors.add(self)

Walls = pygame.sprite.Group()
class Wall(Object):
    def __init__(self, position, graphic = WALL_GRAPHIC):
        super().__init__(position, graphic)
        Walls.add(self)

In [88]:
Sources = pygame.sprite.Group()
class ResourceSource(Wall):
    def __init__(self, position, resource, graphic):
        super().__init__(position, graphic)
        self.resource = resource
        Sources.add(self)
        Walls.add(self)
    
    # take the resource = player now has resource
    def give_resource(self, player):
        player.hands = self.resource(player)
        
    

In [89]:
CounterTops = pygame.sprite.Group()
class CounterTop(Wall):
    def __init__(self, position, graphic = COUNTERTOP_ICON, resource = None):
        super().__init__(position, graphic)
        Walls.add(self)
        CounterTops.add(self)
        self.resource = resource
        
    def put_resource(self, resource):
        self.resource = resource
        resource.place = self
        
    def remove_resouce(self):
        self.resource = None
        
        

In [90]:
CBoards = pygame.sprite.Group()
class CBoard(CounterTop):
    def __init__(self, position, resource = None, graphic = CB_ICON):
        super().__init__(position, graphic)
        self.resource = resource
        self.chopping = False
        self.chopped = False
        self.progress = 0
        self.progress_max = 100
        CBoards.add(self)

    #Remove knife if there was a knife, add a knife if there was no knife
    def adjust_knife(self):
        print("ADJUSTED")
        if(self.image == CB_ICON):
            self.image = CB_KNIFELESS_ICON
            
        elif(self.image == CB_KNIFELESS_ICON):
            self.image = CB_ICON
            
        elif(self.image == CB_R_ICON):
            self.image = CB_R_KNIFELESS_ICON
            
        elif(self.image == CB_R_KNIFELESS_ICON):
            self.image = CB_R_ICON
        
        
    #Overriding to add the removal of a knife
    def put_resource(self, resource):
        super().put_resource(resource)
        self.adjust_knife()
    
    def remove_resouce(self):
        super().remove_resouce()
        self.adjust_knife()
    
    def start_chopping(self):
        self.chopping = True

    def stop_chopping(self):
        self.chopping = False

    def update(self):
        if self.chopping:
            self.progress += 1
            if self.progress >= self.progress_max:
                self.progress = self.progress_max
                self.stop_chopping()
                self.resource.chop() #DEFINE WHAT HAPPENS - CHANGE OF A GRAPHIC AND OF A BOOL

    def draw_progress_bar(self, surface):
        if self.chopping:
            bar_length = 100
            bar_height = 10
            fill = (self.progress / self.progress_max) * bar_length
            pygame.draw.rect(surface, BLACK, [self.rect.x, self.rect.y - 20, bar_length, bar_height], 2)
            pygame.draw.rect(surface, GREEN, [self.rect.x, self.rect.y - 20, fill, bar_height])

In [91]:
Players = pygame.sprite.Group()
class Player(Object):
    def __init__(self, position, player_graphic, controls):
        super().__init__(position, player_graphic)
        self.controls = controls
        self.hands = None
        self.speed = PLAYER_SPEED
        self.HAND_LENGTH = math.sqrt(2)*SIZE
        Players.add(self)
        
    
    def update(self, keys):
        #Actions - grabbing a resource, putting down a reource; only one action can be done in one update
        if keys[self.controls['ACTION']]:
            action_made = False
            for source in Sources:
                if(not action_made):
                    action_made = action_made or self.grab_resource(source)
            
            for countertop in CounterTops:
                if (not action_made):
                    action_made = action_made or self.put_down_resource(countertop)
        
        #Move up down, left, or right
        self.moved = False
        prex, prey = self.rect.x, self.rect.y
        if keys[self.controls['UP']]:
            self.rect.y -= self.speed
            self.moved = True
            
        if keys[self.controls['DOWN']]:
            self.rect.y += self.speed
            self.moved = True
            
        if keys[self.controls['LEFT']]:
            self.rect.x -= self.speed
            self.moved = True
            
        if keys[self.controls['RIGHT']]:
            self.rect.x += self.speed
            self.moved = True
            
        self.last_move = (self.rect.x - prex, self.rect.y - prey)
        if(self.moved):
            self.check_collision()
            
    
    #Get back to the former position if you collided with a wall or a player
    def bounce_back(self):
        self.rect.x, self.rect.y = self.rect.x - self.last_move[0], self.rect.y - self.last_move[1]
        
        
    #Check for collisions with Walls and the other players
    def check_collision(self):
        collisions = pygame.sprite.spritecollide(self, Walls, False)
        if(collisions):
            self.bounce_back()
            
        players = 0
        for player in Players:
            if(self.rect.colliderect(player)):
                players += 1
                
        assert(players > 0)
        if(players > 1):
            self.bounce_back()
    
    #See if the resource source is close enough and if so, grab a resource
    def grab_resource(self, source):
        if ((self.check_distance(source) < self.HAND_LENGTH) & (self.hands == None)):
            source.give_resource(self)
            return True
        
        return False
            
    #See if the countertop is close enough and if so, put down a resource   
    def put_down_resource(self, table):
        if ((self.check_distance(table) < self.HAND_LENGTH) & (self.hands != None) & (table.resource == None)):
            table.put_resource(self.hands)
            self.hands = None
            return True

        return False
    


In [92]:
class ResourceGroup(pygame.sprite.Group):
    #Overriding draw because of a specific nature of resources being drawn
    def draw(self, surface):
        for sprite in self.sprites():
            sprite.determine_position()
        super().draw(surface)
    


Resources = ResourceGroup()
class Resource(pygame.sprite.Sprite):
    def __init__(self, graphics, place):
        self.icon = graphics["icon"]
        self.plain_graphic = graphics["plain"]
        self.place = place
        self.position = None
        self.image = None
        self.rect = None
        super().__init__()
        Resources.add(self)
        
        
    def determine_position(self):
        if(isinstance(self.place, Player)):
            player = self.place
            self.position = (player.rect.center[0] + 20, player.rect.center[1] - 20)
            self.image = self.icon
            self.rect = self.image.get_rect(center = self.position)
            
        if(isinstance(self.place, CounterTop)):
            table = self.place
            self.position = table.rect.center
            self.image = self.plain_graphic
            self.rect = self.image.get_rect(center = self.position)
        
        
        
FISH_GRAPHICS = {
    "icon": FISH_ICON,
    "plain": FISH_PLAIN
}

class Fish(Resource):
    def __init__(self, place):
        super().__init__(FISH_GRAPHICS, place)
    
POTATO_GRAPHICS = {
    "icon": POTATO_ICON,
    "plain": POTATO_PLAIN
}

class Potato(Resource):
    def __init__(self, place):
        super().__init__(POTATO_GRAPHICS, place)

In [93]:
ROWS = 12
COLS = 18

START_X, START_Y = (80, 60)
floor_plan_matrix = [([0]*COLS) for _ in range(ROWS)]
for i in range (COLS):
    floor_plan_matrix[0][i] = 1
    floor_plan_matrix[ROWS-1][i] = 1

for i in range (ROWS):
    floor_plan_matrix[i][0] = 1
    floor_plan_matrix[i][COLS-1] = 1
    
WALL_COLUMN = 11
WALL_LENGTH = 8
for i in range (8):
    floor_plan_matrix[i][WALL_COLUMN] = 1

In [94]:
fish_x, fish_y = (8, 1)
floor_plan_matrix[fish_y][fish_x] = 3
FishCrate = ResourceSource((START_X+SIZE/2 + fish_x*SIZE, START_Y+SIZE/2 + fish_y*SIZE), Fish, FISH_CRATE_ICON)


potato_x, potato_y = (COLS-2, 5)
floor_plan_matrix[potato_y][potato_x] = 3
PotatoCrate = ResourceSource((START_X+SIZE/2 + potato_x*SIZE, START_Y+SIZE/2 + potato_y*SIZE), Potato, POTATO_CRATE_ICON)


CB1_x, CB1_y = fish_x-1, fish_y
floor_plan_matrix[CB1_y][CB1_x] = 3
CBoard1 = CBoard((START_X+SIZE/2 + CB1_x*SIZE, START_Y+SIZE/2 + CB1_y*SIZE))


CB2_x, CB2_y = potato_x, potato_y+1
floor_plan_matrix[CB2_y][CB2_x] = 3
CBoard2 = CBoard((START_X+SIZE/2 + CB2_x*SIZE, START_Y+SIZE/2 + CB2_y*SIZE), None, CB_R_ICON)

In [95]:
obj_types = [[Floor, Floors], [Wall, Walls]]
for i in range (ROWS):
    for j in range (COLS):
        if(floor_plan_matrix[i][j] < 2):
            obj_class, obj_sprite = obj_types[floor_plan_matrix[i][j]]
            object = obj_class([START_X+SIZE/2 + j*SIZE, START_Y+SIZE/2 + i*SIZE])
            obj_sprite.add(object)



player1_controls = {
    "UP": pygame.K_w,
    "DOWN": pygame.K_s,
    "LEFT": pygame.K_a,
    "RIGHT": pygame.K_d,
    "ACTION": pygame.K_e
}

player2_controls = {
    "UP": pygame.K_UP,
    "DOWN": pygame.K_DOWN,
    "LEFT": pygame.K_LEFT,
    "RIGHT": pygame.K_RIGHT,
    "ACTION": pygame.K_SPACE
}

Player1 = Player([200, 120], PLAYER1_GRAPHIC, player1_controls)
Players.add(Player1)

Player2 = Player([500, 120], PLAYER2_GRAPHIC, player2_controls)
Players.add(Player2)

In [96]:
running = True
clock = pygame.time.Clock()


while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            pygame.quit()

    screen.fill((255, 255, 255)) 

    Players.update(pygame.key.get_pressed())
    if pygame.QUIT in pygame.event.get():
        running = False
        
    Floors.draw(screen)
    Walls.draw(screen)
    Players.draw(screen)
    Resources.draw(screen)

    pygame.display.flip()
    clock.tick(60)
D
        

# Quit Pygame
pygame.quit()


ADJUSTED
ADJUSTED


error: display Surface quit

: 